In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

embed_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

texts = rag_df["text"].astype(str).tolist()
emb = embed_model.encode(texts, batch_size=64, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=True)

index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)
print("FAISS vectors:", index.ntotal)


---

secondary'den pozitif örnekler (monitor/avoid)

df_pos = df_secondary[df_secondary["action_3class"].isin(["monitor","avoid"])][["drugA_norm","drugB_norm","pair_key","action_3class"]].dropna().drop_duplicates()

negatif üretim: random drug havuzundan "hiçbir kaynakta görünmeyen" çiftler seç

all_drugs = sorted(set(df_secondary["drugA_norm"].dropna().tolist() + df_secondary["drugB_norm"].dropna().tolist()))
pos_keys = set(df_pos["pair_key"].tolist())

def generate_no_interaction_pairs(n=3000, max_tries=200000):
rng = np.random.default_rng(42)
out = []
tries = 0
out_keys = set()

while len(out) < n and tries < max_tries:
    tries += 1
    a = all_drugs[int(rng.integers(0, len(all_drugs)))]
    b = all_drugs[int(rng.integers(0, len(all_drugs)))]
    if a == b:
        continue
    key = make_pair_key(a, b)
    if key in pos_keys or key in out_keys:
        continue


    out.append({"drugA_norm": a, "drugB_norm": b, "pair_key": key, "action_3class": "no_interaction"})
    out_keys.add(key)

return pd.DataFrame(out)

df_no = generate_no_interaction_pairs(3000)
print("no_interaction:", df_no.shape)

df_model = pd.concat([df_pos, df_no], ignore_index=True).drop_duplicates("pair_key")
print(df_model["action_3class"].value_counts())

🔴 TEXT KOLONUNU GARANTİ ALTINA AL

df_model = df_model.copy()

df_model["text"] = (
"Interaction between " +
df_model["drugA_norm"].astype(str) +
" and " +
df_model["drugB_norm"].astype(str)
)

Label zaten vardı ama garanti edelim

label_names = ["no_interaction", "monitor", "avoid"]
label2id = {n: i for i, n in enumerate(label_names)}
df_model["label"] = df_model["action_3class"].map(label2id)

Kontrol

print(df_model[["text", "action_3class", "label"]].head())
print(df_model["action_3class"].value_counts())

from datasets import Dataset
from sklearn.model_selection import train_test_split

label_names = ["no_interaction","monitor","avoid"]
label2id = {n:i for i,n in enumerate(label_names)}
id2label = {i:n for n,i in label2id.items()}

df_model["label"] = df_model["action_3class"].map(label2id)

train_df, test_df = train_test_split(
df_model, test_size=0.3, random_state=42, stratify=df_model["label"]
)

ds_train = Dataset.from_pandas(train_df[["text","label"]])
ds_test  = Dataset.from_pandas(test_df[["text","label"]])

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

MODEL_BASE = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_BASE)

def tok(batch):
return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

ds_train = ds_train.map(tok, batched=True)
ds_test  = ds_test.map(tok, batched=True)

ds_train.set_format(type="torch", columns=["input_ids","attention_mask","label"])
ds_test.set_format(type="torch", columns=["input_ids","attention_mask","label"])

model = AutoModelForSequenceClassification.from_pretrained(
MODEL_BASE,
num_labels=len(label_names),
id2label=id2label,
label2id=label2id
)

class weights (dengesizliği azaltır)

counts = train_df["label"].value_counts().sort_index().values
weights = (counts.sum() / (len(counts) * counts))
class_weights = torch.tensor(weights, dtype=torch.float)

class WeightedTrainer(Trainer):
def compute_loss(self, model, inputs, return_outputs=False, kwargs):
labels = inputs.get("labels") if "labels" in inputs else inputs.get("label")
if labels is None:
labels = inputs["label"]
outputs = model({k:v for k,v in inputs.items() if k in ["input_ids","attention_mask"]})
logits = outputs.logits
loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
loss = loss_fct(logits, labels)
return (loss, outputs) if return_outputs else loss

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
logits, labels = eval_pred
preds = np.argmax(logits, axis=-1)
return {
"accuracy": accuracy_score(labels, preds),
"f1_macro": f1_score(labels, preds, average="macro"),
"precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
"recall_macro": recall_score(labels, preds, average="macro"),
}

args = TrainingArguments(
output_dir="drive/MyDrive/İlaç/content/biobert_ddi_3class_secondary2",
learning_rate=2e-5,
per_device_train_batch_size=8,
per_device_eval_batch_size=8,
num_train_epochs=8,
evaluation_strategy="epoch",
save_strategy="epoch",
logging_steps=50,
report_to="none",
)

trainer = WeightedTrainer(
model=model,
args=args,
train_dataset=ds_train,
eval_dataset=ds_test,
tokenizer=tokenizer,
compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_model = AutoTokenizer.from_pretrained(MODEL_PATH)
model_ddi = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

model_ddi.to(device)
model_ddi.eval()

print("✅ Model başarıyla yüklendi:", MODEL_PATH)
